In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/18"
pmc = "15022006" # 店舗ID
SPREADSHEET_KEY = '1VL-BM8KP-jfr9qcBdqWu8RpHAhln7FXR0nkHzeibo6M' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 770 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+456@gmail.com"
user_pw = "paruparu22"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "112":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "112":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/18', '025739', '1', '4359.0', '-3309.0', '2', '2']
['08/18', '025739', '2', '4615.0', '18056.0', '18', '2']
['08/18', '025825', '3', '9487.0', '-8124.0', '0', '0']
['08/18', '025667', '5', '1282.0', '-1203.0', '0', '0']
['08/18', '025549', '6', '0', '0', '0', '0']
['08/18', '025483', '7', '0', '0', '0', '0']
['08/18', '025501', '8', '20256.0', '2408.0', '39', '12']
['08/18', '025285', '10', '9487.0', '1506.0', '13', '6']
['08/18', '025165', '11', '4615.0', '603.0', '6', '1']
['08/18', '025529', '12', '8205.0', '-6018.0', '1', '1']
['08/18', '025523', '13', '9744.0', '7524.0', '19', '7']
['08/18', '025472', '15', '15385.0', '-10231.0', '4', '3']
['08/18', '025474', '16', '3590.0', '302.0', '5', '1']
['08/18', '025500', '17', '7436.0', '1806.0', '15', '4']
['08/18', '025440', '18', '8462.0', '1205.0', '10', '3']
['08/18', '025385', '20', '8462.0', '904.0', '12', '4']
['08/18', '025419', '21', '3333.0', '-902.0', '3', '1']
['08/18', '025460', '22', '17436.0', '-7221.0', '12', '8']
['

['08/18', '025227', '251', '10513.0', '8427.0', '31', '2']
['08/18', '025688', '252', '4872.0', '-3610.0', '1', '1']
['08/18', '025473', '253', '4359.0', '302.0', '4', '1']
['08/18', '025670', '255', '11795.0', '-902.0', '20', '5']
['08/18', '025417', '256', '12308.0', '302.0', '14', '4']
['08/18', '025647', '257', '15897.0', '-1203.0', '27', '6']
['08/18', '025677', '258', '18974.0', '-3008.0', '10', '3']
['08/18', '025677', '260', '4103.0', '-3610.0', '0', '0']
['08/18', '025677', '261', '12821.0', '904.0', '9', '3']
['08/18', '025677', '262', '2564.0', '-2406.0', '0', '0']
['08/18', '025677', '263', '14359.0', '11436.0', '20', '5']
['08/18', '025677', '265', '13590.0', '8126.0', '15', '2']
['08/18', '025837', '266', '2564.0', '-1805.0', '1', '1']
['08/18', '025837', '267', '1026.0', '-902.0', '0', '0']
['08/18', '025665', '268', '24615.0', '-3309.0', '12', '2']
['08/18', '025665', '270', '13077.0', '-3008.0', '7', '4']
['08/18', '025665', '271', '15897.0', '-13541.0', '0', '0']
['08

['08/18', '025856', '573', '14872.0', '-9930.0', '2', '1']
['08/18', '025764', '575', '0', '0', '0', '0']
['08/18', '025764', '576', '1795.0', '7524.0', '6', '1']
['08/18', '025764', '577', '6410.0', '-3911.0', '1', '1']
['08/18', '025764', '578', '0', '0', '0', '0']
['08/18', '025764', '580', '513.0', '-902.0', '0', '0']
['08/18', '025818', '581', '513.0', '-902.0', '0', '0']
['08/18', '025818', '582', '0', '0', '0', '0']
['08/18', '025818', '583', '16154.0', '-13842.0', '0', '0']
['08/18', '025829', '585', '3333.0', '4515.0', '5', '2']
['08/18', '025829', '586', '3846.0', '-3309.0', '0', '0']
['08/18', '025779', '587', '3846.0', '-2707.0', '2', '2']
['08/18', '025779', '588', '5897.0', '-4513.0', '1', '1']
['08/18', '025646', '600', '1026.0', '-902.0', '0', '0']
['08/18', '025707', '601', '1538.0', '-1504.0', '0', '0']
['08/18', '025707', '602', '1282.0', '-1203.0', '0', '0']
['08/18', '025789', '603', '2564.0', '-2106.0', '0', '0']
['08/18', '025789', '605', '2564.0', '-2406.0', '0'

['08/18', '119852', '811', '0', '0', '0', '0', '--']
['08/18', '119982', '812', '723', '93.0', '4', '1', '--']
['08/18', '119982', '813', '0', '0', '0', '0', '--']
['08/18', '119945', '815', '0', '0', '0', '0', '--']
['08/18', '119945', '816', '0', '0', '0', '0', '--']
['08/18', '120043', '817', '560', '-231.0', '1', '2', '--']
['08/18', '119929', '818', '0', '0', '0', '0', '--']
['08/18', '120033', '820', '2506', '-972.0', '6', '4', '--']
['08/18', '120086', '821', '830', '509.0', '6', '3', '--']
['08/18', '120086', '822', '0', '0', '0', '0', '--']
['08/18', '120086', '823', '106', '-93.0', '0', '0', '--']
['08/18', '120017', '825', '30', '0', '0', '0', '--']
['08/18', '120017', '826', '0', '0', '0', '0', '--']
['08/18', '119972', '827', '388', '139.0', '1', '1', '--']
['08/18', '119972', '828', '0', '0', '0', '0', '--']
['08/18', '119984', '830', '2536', '-93.0', '10', '7', '--']
['08/18', '119984', '831', '2816', '972.0', '14', '7', '--']
['08/18', '119984', '832', '0', '0', '0', '0

['08/18', '120110', '1130', '4123', '-972.0', '21', '13', '--']
['08/18', '120110', '1131', '409', '-556.0', '0', '0', '--']
['08/18', '120110', '1132', '2858', '4352.0', '32', '22', '--']
['08/18', '120110', '1133', '3220', '-93.0', '19', '13', '--']
['08/18', '120110', '1135', '552', '463.0', '5', '2', '--']
['08/18', '120110', '1136', '1796', '2639.0', '22', '7', '--']
['08/18', '120021', '1137', '0', '0', '0', '0', '--']
['08/18', '120021', '1138', '0', '0', '0', '0', '--']
['08/18', '120021', '1150', '0', '0', '0', '0', '--']
['08/18', '120021', '1151', '0', '0', '0', '0', '--']
['08/18', '120021', '1152', '0', '0', '0', '0', '--']
['08/18', '120021', '1153', '0', '0', '0', '0', '--']
['08/18', '120021', '1155', '52', '0', '0', '0', '--']
['08/18', '120021', '1156', '0', '0', '0', '0', '--']
['08/18', '120021', '1157', '0', '0', '0', '0', '--']
['08/18', '120080', '1158', '1603', '-694.0', '9', '0', '13']
['08/18', '119956', '1160', '1289', '-926.0', '0', '2', '2']
['08/18', '1199

{'spreadsheetId': '1VL-BM8KP-jfr9qcBdqWu8RpHAhln7FXR0nkHzeibo6M',
 'updates': {'spreadsheetId': '1VL-BM8KP-jfr9qcBdqWu8RpHAhln7FXR0nkHzeibo6M',
  'updatedRange': 'S20RAW!F43301:M43555',
  'updatedRows': 255,
  'updatedColumns': 8,
  'updatedCells': 2040}}